In [ ]:
import torch
import torch.nn as nn

from torch.utils.tensorboard import SummaryWriter

In [17]:
import sys
sys.path.append('/home/raquelpanadero/Desktop/TFG/scripts')

from utils import create_dataloaders
from models.bb_encoder_decoder import BBEncoderDecoder
from trainers.bb_trainer import train_model

In [3]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [4]:
data = torch.load('/home/raquelpanadero/Desktop/TFG/waymo_dataset_images/dataset.pt')

## Training

In [ ]:
list_batch_sizes = [16, 32, 64, 128, 256, 512]
list_lr = [0.01, 0.001, 0.0001, 1e-5, 1e-6]

# define the net
input_size = 8
output_size = 4
hidden_size = 64
num_layers = 2
drop_prob = 0.5
target_len = 9
n_epochs = 25
device = 'cuda' if torch.cuda.is_available() else 'cpu'

for batch_size in list_batch_sizes:
    for lr in list_lr:
        train_loader, valid_loader, test_loader = create_dataloaders(data, batch_size=batch_size)
        writer = SummaryWriter(
            log_dir=('/home/raquelpanadero/Desktop/TFG/logs'
                     f'/bb_ed_lr_{lr}_bsize_{batch_size}')
        )
        net = BBEncoderDecoder(
            input_size=input_size,
            output_size=output_size, 
            batch_size=batch_size, 
            target_len=target_len, 
            hidden_size=hidden_size,
            num_layers=num_layers,
            device=device
        )
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(net.parameters(), lr=lr)

        train_model(
            net = net,
            train_loader = train_loader,
            valid_loader = valid_loader,
            n_epochs = n_epochs, 
            optimizer = optimizer,
            criterion = criterion,
            device = device,
            writer=writer
        )

## Model evaluation

In [ ]:
%tensorboard --logdir /home/raquelpanadero/Desktop/TFG/logs